Get clean data.

Make sliders for time periods.



In [3]:
import pandas as pd 
import re

data = pd.read_csv('companies.csv')
print(data.head())

def clean_and_convert(value):
    if pd.isna(value) or "N/A" in value.upper():
        return [pd.NA, pd.NA]

    # Remove unwanted characters but keep the hyphen if it's surrounded by spaces (which indicates a range)
    value = re.sub(r'[^0-9.-]+', '', value.replace(' - ', '-').upper())
    
    # Split the value by hyphen if it's a range, otherwise put it in a single-element list
    numbers = value.split('-') if '-' in value else [value]

    # Remove any remaining non-numeric characters and convert to float
    min_val = float(re.sub(r'[^0-9.]+', '', numbers[0])) if numbers[0] else pd.NA
    max_val = float(re.sub(r'[^0-9.]+', '', numbers[1])) if len(numbers) > 1 and numbers[1] else min_val
    
    return [min_val, max_val]


data['cleaned_cost'] = data['Cost'].apply(clean_and_convert)

# Split the 'cleaned_cost' into two separate columns
data[['min_cost', 'max_cost']] = pd.DataFrame(data['cleaned_cost'].tolist(), index=data.index)

# Drop the temporary 'cleaned_cost' column
data.drop('cleaned_cost', axis=1, inplace=True)
data.dropna(subset=['min_cost', 'max_cost'], how='all', inplace=True)

# Display the result
print(data[['min_cost', 'max_cost']])




                    Name                                           Location  \
0     Perle Legacy Homes  Langley, Abbotsford, Delta, Maple Ridge, Surre...   
1     Mr Build Vancouver                                        Shaughnessy   
2  Tessella Construction                                           Squamish   
3         Paragon Valley  Aldergrove, Fraser Valley, Surrey, White Rock,...   
4            Vansa Renos              Coquitlam, Vancouver, North Vancouver   

                         Cost  
0                         NaN  
1                $13K (+gst)   
2                         NaN  
3  CAD 10,000 - CAD 1,000,000  
4       CAD 5,000 - 6 million  
    min_cost   max_cost
1       13.0       13.0
3    10000.0  1000000.0
4     5000.0        6.0
5    20000.0  5000000.0
6    20000.0   500000.0
..       ...        ...
110  50000.0    50000.0
111  25000.0  1000000.0
113    500.0   150000.0
114  15000.0   600000.0
115   5000.0  1000000.0

[88 rows x 2 columns]


/var/folders/cz/570p77r57bl87g4_l6l7xqw00000gn/T/ipykernel_22330/3235195242.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [4]:
expanded_rows = []

# Iterate over each row in the original DataFrame
for index, row in data.iterrows():
    # Split the 'Location' column based on commas
    locations = row['Location'].split(', ')
    # For each location, create a new row with the company, the individual location, and the cost range
    for location in locations:
        expanded_rows.append({
            'Company': row['Name'],
            'Location': location.strip(),  # Strip any leading/trailing whitespace
            'min_cost': row['min_cost'],
            'max_cost': row['max_cost']
        })
        
expanded_df = pd.DataFrame(expanded_rows)
print(expanded_df.head())


              Company       Location  min_cost   max_cost
0  Mr Build Vancouver    Shaughnessy      13.0       13.0
1      Paragon Valley     Aldergrove   10000.0  1000000.0
2      Paragon Valley  Fraser Valley   10000.0  1000000.0
3      Paragon Valley         Surrey   10000.0  1000000.0
4      Paragon Valley     White Rock   10000.0  1000000.0


In [22]:
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import pandas as pd

# Initialize geocoder
geolocator = Nominatim(user_agent="my_geocoder")
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)

# Function to geocode a location
def geocode_location(location):
    try:
        # Attempt to geocode the location
        geocoded_location = geocode(f"{location}, BC, Canada",timeout = 5)
        return geocoded_location.latitude, geocoded_location.longitude
    except:
        # Return NA values if geocoding fails
        return pd.NA, pd.NA

# Apply geocoding to each location in the DataFrame
expanded_df['coords'] = expanded_df['Location'].apply(geocode_location)
expanded_df[['latitude', 'longitude']] = pd.DataFrame(expanded_df['coords'].tolist(), index=expanded_df.index)


NotImplementedError: initializing a Series from a MultiIndex is not supported

In [6]:
# Group by location and compute the mean of the min and max costs
aggregated_df = expanded_df.groupby('Location').agg({
    'min_cost': 'mean',
    'max_cost': 'mean',
    'latitude': 'first',  # Since all latitudes for the same location should be equal
    'longitude': 'first'  # Ditto for longitudes
}).reset_index()

expanded_df = expanded_df[expanded_df['Location'] != 'Su']


In [7]:
# Normalize cost data to range from 0 to 1 for heat map intensity
max_cost = aggregated_df['max_cost'].max()
aggregated_df['cost_weight'] = aggregated_df['max_cost'] / max_cost
import folium
from folium.plugins import HeatMap

# Create a map centered around BC, Canada
map_center = aggregated_df[['latitude', 'longitude']].dropna().mean().tolist()
map = folium.Map(location=map_center, zoom_start=6)

# List of points for the heat map: lat, lng, and the weight
heat_data = [
    (row['latitude'], row['longitude'], row['cost_weight']) 
    for index, row in aggregated_df.iterrows()
    if pd.notna(row['latitude']) and pd.notna(row['longitude'])
]

# Add the heat map layer
HeatMap(heat_data).add_to(map)

# Save to an HTML file
map.save('heat_map.html')


In [ ]:
# Assume 'expanded_df' is already loaded and includes latitude and longitude
# Calculate the average cost
expanded_df['average_cost'] = (expanded_df['min_cost'] + expanded_df['max_cost']) / 2
import plotly.graph_objects as go

# Create the 3D scatter plot
fig = go.Figure(data=[go.Scatter3d(
    x=expanded_df['longitude'],
    y=expanded_df['latitude'],
    z=expanded_df['average_cost'],
    mode='markers',
    marker=dict(
        size=10,
        color=expanded_df['average_cost'],    # Set color according to average cost
        colorscale='Viridis',                 # Color scale for visualization
        opacity=0.8
    ),
    text=expanded_df['Company']              # Hover text
)])

# Update plot layout
fig.update_layout(
    title='3D Plot of Companies by Location and Average Cost',
    scene=dict(
        xaxis_title='Longitude',
        yaxis_title='Latitude',
        zaxis_title='Average Cost'
    ),
    margin=dict(l=0, r=0, b=0, t=0)          # Tight layout
)

# Show the plot
fig.show()

# Optionally, save the plot to an HTML file
fig.write_html('3D_scatter_plot.html')

expanded_df.to_csv('processed_companies.csv', index=False)



In [21]:
import plotly.express as px
expanded_df['Location'] = expanded_df['Location'].astype('category')
expanded_df['Company'] = expanded_df['Company'].astype('category')

# Normalize the cost data
expanded_df['min_cost_normalized'] = (expanded_df['min_cost'] - expanded_df['min_cost'].min()) / (expanded_df['min_cost'].max() - expanded_df['min_cost'].min())
expanded_df['max_cost_normalized'] = (expanded_df['max_cost'] - expanded_df['max_cost'].min()) / (expanded_df['max_cost'].max() - expanded_df['max_cost'].min())

fig = px.parallel_coordinates(expanded_df, 
                              dimensions=['Company', 'Location', 'min_cost_normalized', 'max_cost_normalized'],
                              labels={'min_cost_normalized': 'Minimum Cost',
                                      'max_cost_normalized': 'Maximum Cost',
                                      'Location': 'Location',
                                      'Company': 'Company Name'},
                              color='min_cost_normalized',
                              color_continuous_scale=px.colors.diverging.Tealrose,
                              title='Parallel Coordinates: Company Costs by Location')
fig.show()


KeyError: 'Company'